<img src = 'https://bancariosbh.org.br/manager/resources/uploads/213/350_198_8e66ca00d7dfdfbd00096a7c220be22c.jpg' height="200" width="340">
<p><b><font size = 5 face = 'verdana'> TAREFA AVALIATIVA 3 - PROCESSAMENTO DE LINGUAGEM NATURAL</font></b></p>

<p><font size = 3 face = 'verdana'>Trabalho apresentado à conclusão do curso de PROCESSAMENTO DE LINGUAGEM NATURAL do curso de Pós-Graduação em Inteligência Artificial, PUC Minas, como requisito parcial de avaliação. </font></b></p>
<p><font size = 3 face = 'verdana'><b>PROFESSORA:</b> BARBARA SILVEIRA FRAGA<br> 
<b>ALUNO:</b> OSEMAR DA SILVA XAVIER<br>
<b>MATRÍCULA:</b> 1418586</font></p>
<p><font size = 3 face = 'verdana'>
<hr></hr>

In [166]:
!pip install afinn
!python -m textblob.download_corpora
!pip install -U textblob
!pip install vaderSentiment
!pip install pyemd
!pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /root/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
Finished.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: h

In [258]:
import re
import nltk
from nltk.util import ngrams
from nltk.corpus import stopwords
import string
from unidecode import unidecode
import pandas as pd
import bz2
import gensim
import warnings
import numpy as np
from gensim.models import word2vec
from sklearn.feature_extraction.text import CountVectorizer
from scipy.spatial import distance
from sklearn.metrics.pairwise import cosine_similarity
from tqdm._tqdm_notebook import tqdm_notebook
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score

warnings.filterwarnings('ignore')
tqdm_notebook.pandas()

import matplotlib.pyplot as plt
%matplotlib inline

# Carregando os embeddings

Aqui vamos utilizar os embeddings para realizar as seguintes atividades:

- análise de simlaridade
- classificação de documentos

<b> Carregue os embeddings treinados, como vimos na Aula 2. É o mesmo arquivo que iremos utilizar</b>

Link: https://drive.google.com/open?id=1zI8pGfbUHuU_0wY_FV4tD6w6ZCUJTQbh

In [168]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [169]:
#opção 1 -> montar o drive no colab e acessar o arquivo de embedding do drive
from google.colab import drive
drive.mount('/content/drive')

#opção 2 -> fazer download e fazer upload por aqui
#from google.colab import files
#uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [170]:
%%time
# carregar
##na variável path coloque o caminho do embedding baixado:
path = "/content/drive/MyDrive/PUC Minas/Materias/PROCESSAMENTO DE LINGUAGEM NATURAL/Aula 3/ptwiki_20180420_100d.txt"
word_vectors = gensim.models.KeyedVectors.load_word2vec_format(path, binary=False)

CPU times: user 1min 8s, sys: 2.19 s, total: 1min 11s
Wall time: 1min 19s


# Similaridade de Documentos

Para realizar a similaridade entre documentos, utilize as frases abaixo:

In [171]:
frase1 = "Excelente produto chegou antes do prazo indico e recomendo produto bom pois já testei e foi mais que aprovado" 
frase2 = "SUPER RECOMENDO, PREÇO, QUALIDADE #BRASTEMP, EFICIÊNCIA NA ENTREGA, E FACILIDADE DE PAGAMENTO. MUITO BOM!!!"
frase3 = "A tampa do fogão veio com problemas com o pino de encaixe solto e precisa de reparos"
frase4 = "Fogão ótimo!"

## Distância de Jaccard

<b> Atividade </b>

1) Faça um método que calcule a similaridade de Jaccard e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: lembrando que você precisa aplicar um pre-processamento nessas frases antes de aplicar o método.
Faça:

- Lower
- Remoção StopWords
- Remoção Pontuação
- Tokenização

In [172]:
# Foi necessário fazer a importação do sys porque estava dando erro 
# 'unicode' is not defined
import sys
if sys.version_info[0] >= 3:
    unicode = str

In [173]:
# Fiz o download das stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [174]:
# Fazendo o pré processamento
def pre_processamento_texto(corpus):
  corpus_alt = re.findall(r'\w+(?:\w+)?|[^\w\s]', corpus)
  corpus_alt = [t.lower() for t in corpus_alt]
  portugues_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  corpus_alt =[t for t in corpus_alt if t not in string.punctuation]
  corpus_alt = [unicode(t) for t in corpus_alt]
  return corpus_alt

In [175]:
# Criei uma variável para cada texto
frase_limpa1 = pre_processamento_texto(frase1)
frase_limpa2 = pre_processamento_texto(frase2)
frase_limpa3 = pre_processamento_texto(frase3)
frase_limpa4 = pre_processamento_texto(frase4)

In [176]:
frase_limpa4

['fogão', 'ótimo']

In [177]:
# Com o set é possível criar intercessão entre essas 
# frases que foram pré processadas. Por exemplo:
set(frase_limpa1).intersection(set(frase_limpa2))

{'bom', 'recomendo'}

In [178]:
# Dessa forma, para calcular a intercessão entre as duas frases é: 
print('A intercessão entre a frase 1 e 2 é ',len(set(frase_limpa1).intersection(set(frase_limpa2)))/len(set(frase_limpa1).union(set(frase_limpa2))))

A intercessão entre a frase 1 e 2 é  0.10526315789473684


In [179]:
# Dessa forma, para calcular a intercessão entre as duas frases é: 
print('A intercessão entre a frase 1 e 3 é ',len(set(frase_limpa1).intersection(set(frase_limpa3)))/len(set(frase_limpa1).union(set(frase_limpa3))))

A intercessão entre a frase 1 e 3 é  0.0


In [180]:
# Dessa forma, para calcular a intercessão entre as duas frases é: 
print('A intercessão entre a frase 2 e 3 é ',len(set(frase_limpa2).intersection(set(frase_limpa3)))/len(set(frase_limpa2).union(set(frase_limpa3))))

A intercessão entre a frase 2 e 3 é  0.0


In [181]:
# Dessa forma, para calcular a intercessão entre as duas frases é: 
print('A intercessão entre a frase 1 e 4 é ',len(set(frase_limpa1).intersection(set(frase_limpa4)))/len(set(frase_limpa1).union(set(frase_limpa4))))

A intercessão entre a frase 1 e 4 é  0.0


In [182]:
# Uma forma mais prática de fazer é utilizando o método abaixo:
def jaccard_similarity(list1, list2):
  s1 = set(list1)
  s2 = set(list2)
  return len(s1.intersection(s2))/ len(s1.union(s2))

In [183]:
# Imprimindo o resultado das intercessões 
print('A intercessão entre a frase 1 e 2 é ',jaccard_similarity(frase_limpa1, frase_limpa2))
print('A intercessão entre a frase 1 e 3 é ',jaccard_similarity(frase_limpa1, frase_limpa3))
print('A intercessão entre a frase 2 e 3 é ',jaccard_similarity(frase_limpa2, frase_limpa3))
print('A intercessão entre a frase 1 e 4 é ',jaccard_similarity(frase_limpa1, frase_limpa4))

A intercessão entre a frase 1 e 2 é  0.10526315789473684
A intercessão entre a frase 1 e 3 é  0.0
A intercessão entre a frase 2 e 3 é  0.0
A intercessão entre a frase 1 e 4 é  0.0


<b> Atividade </b>

2) Qual par de frase teve maior simlaridade? E qual teve menor? Este resultado faz sentido? Explique.<br>
**R.: Para o exemplo que foi dado, a frase 1 e 2 teve similaridade. O restante não possui similaridade, pois seus resultados foram igual a zero. Deveria ter dado similiradade mas o método utilizado não foi suficiente, por ser limitado. É recomendado que utilize outros métodos para melhorar esse resultado.** 

## Distância de Cosseno

Aqui iremos calcular a distância do cosseno utilizando duas formas, que aprendemos na aula passada, para representar o texto.

- Bag of Words (BOW) 
- Embedding

Observação:

Existem duas formas de trabalhar com o cosseno:

<b> Distância </b>: quanto menor mais perto estão as frases.
<b> Similaridade </b>: quanto maior mais perto estão as frases.

### BOW - Distância do cosseno

<b> Atividade </b>

3) Calcule a distância do cosseno utilizando a representação CountVectorizer e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: no CountVectorizer utilizem as frases já pre-processadas da atividade anterior. Mas para aplicá-las no fit_transform, cada frase deve ser um string (sem estar tokenizada) dentro de uma lista.

```python
#exemplo
distance.cosine(frase1, frase2)
```

In [184]:
# Criando um objeto com CountVectorize
vect_bag = CountVectorizer() 

In [185]:
# Criando o vetor, mas dessa forma não vai funcionar pq é uma lista
# dessa forma é necessário transformar em uma lista de documentos (lista de string)
# 'list' object has no attribute 'lower'
#vetor = vect_bag.fit_transform([frase_limpa1,
#                        frase_limpa2,
#                        frase_limpa3,
#                        frase_limpa4])

In [186]:
vetor = vect_bag.fit_transform([' '.join(frase_limpa1),
                        ' '.join(frase_limpa2),
                        ' '.join(frase_limpa3),
                        ' '.join(frase_limpa4)])

In [187]:
frase_1_bow = vetor.todense()[0]
frase_2_bow = vetor.todense()[1]
frase_3_bow = vetor.todense()[2]
frase_4_bow = vetor.todense()[3]

In [188]:
print('Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 1 e frase 2 é ',distance.cosine(frase_1_bow,frase_2_bow))
print('Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 1 e frase 3 é ',distance.cosine(frase_1_bow,frase_3_bow))
print('Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 2 e frase 3 é ',distance.cosine(frase_2_bow,frase_3_bow))
print('Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 1 e frase 4 é ',distance.cosine(frase_1_bow,frase_4_bow))

Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 1 e frase 2 é  0.8309691490542968
Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 1 e frase 3 é  1.0
Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 2 e frase 3 é  1.0
Utilizando o método BOW para calcular distância do cosseno a similaridade entre a frase 1 e frase 4 é  1.0


### Embedding - Distância do cosseno

Para calcular o embedding de cada uma das frases, utilize o modelo carregado inicialmente. 

Cada palavra tem um vetor, para formar o embedding da frase tire a média de todos os vetores.

Utilize as frases já pre-processadas

<b> Atividade </b> 

4) Calcule a distância do cosseno utilizando a representação Embedding e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4


In [189]:
frase_limpa4

['fogão', 'ótimo']

In [190]:
# Embedding abaixo foi feito a média entre as duas matrizes, considerando
# 'fogão' e 'ótimo' 
np.mean([word_vectors['fogão'], word_vectors['ótimo']], axis=0)

array([ 0.18785   ,  0.126     ,  0.0847    , -0.06649999, -0.51      ,
       -0.18645   ,  0.024     ,  0.36014998, -0.35944998,  0.60525   ,
        0.44535   , -0.18255   , -0.11875001, -0.0331    , -0.1221    ,
        0.2004    ,  0.18325001, -0.56184995,  0.0987    ,  0.0876    ,
       -0.3509    ,  0.1683    , -0.21724999, -0.12375   , -0.2542    ,
        0.12035   , -0.35180002, -0.6065    , -0.09224999, -0.10515001,
        0.38375   ,  0.13495   ,  0.6185    ,  0.7058    , -0.15475   ,
       -0.5147    , -0.04935   , -0.23615001,  0.39105   ,  0.1099    ,
        0.19095   , -0.12635   , -0.4124    ,  0.1274    ,  0.2784    ,
       -0.00785   , -0.16675   ,  0.0574    , -0.8424    , -0.24395   ,
        0.81105   ,  0.0604    , -0.23394999,  0.08015   ,  0.33105   ,
       -0.1822    , -0.00505   , -0.68729997,  0.34364998,  0.34710002,
        0.52085   , -0.68044996, -0.1737    ,  0.1291    ,  0.2587    ,
        0.20009999, -0.16925   ,  0.0454    ,  0.2423    ,  0.18

In [191]:
# Utilizando função para calcular o embedding da frase
def calcula_embedding_frase(tokens):
	return np.mean(np.array([word_vectors[t] for t in tokens if t in word_vectors.vocab]), axis=0)

In [192]:
# Calculando o embedding para cada frase
frase_1_emb = calcula_embedding_frase(frase_limpa1)
frase_2_emb = calcula_embedding_frase(frase_limpa2)
frase_3_emb = calcula_embedding_frase(frase_limpa3)
frase_4_emb = calcula_embedding_frase(frase_limpa4)

In [193]:
# Já que tenho os embeddings de cada frase, agora posso calcular o cosseno
# Considerando os valores abaixo, e a regra de quanto mais próximo de zero
# maior a similaridade. O resultado foi mais satisfatório.
print(distance.cosine(frase_1_emb,frase_2_emb))
print(distance.cosine(frase_1_emb,frase_3_emb))
print(distance.cosine(frase_2_emb,frase_3_emb))
print(distance.cosine(frase_1_emb,frase_4_emb))

0.16096973419189453
0.2234203815460205
0.2526938319206238
0.29043054580688477


<b>Atividade </b>

5) Qual diferença entre as representações do exercício 3 e 4? Qual faz mais sentido? Explique.<br>
***R: Tanto no BOW quanto o Embedding são utilizados a distância do cosseno, porém quando se usa uma representação textual melhor, a tendência do resultado  é ser melhor. ***

## WMD

O WMD já está incorporado ao Word2Vec

<b> Atividade </b>

6) Calcule a distância WMD e aplique para os seguintes pares de frases:

- Frase1 e Frase2
- Frase1 e Frase3
- Frase2 e Frase3
- Frase1 e Frase4

Observação: use a variável já tokenizada.

In [194]:
# A vantagem do WMD é que não é necessário fazer o pré processamento
# Pode-se usar a forma textual normal
print('Utilizando o método WMD o resultado para a frase 1 e frase 2 é ', word_vectors.wmdistance(frase_limpa1, frase_limpa2))
print('Utilizando o método WMD o resultado para a frase 1 e frase 3 é ', word_vectors.wmdistance(frase_limpa1, frase_limpa3))
print('Utilizando o método WMD o resultado para a frase 2 e frase 3 é ', word_vectors.wmdistance(frase_limpa2, frase_limpa3))
print('Utilizando o método WMD o resultado para a frase 1 e frase 4 é ', word_vectors.wmdistance(frase_limpa1, frase_limpa4))

Utilizando o método WMD o resultado para a frase 1 e frase 2 é  3.0725697404938526
Utilizando o método WMD o resultado para a frase 1 e frase 3 é  3.759219170455064
Utilizando o método WMD o resultado para a frase 2 e frase 3 é  3.9048993635316385
Utilizando o método WMD o resultado para a frase 1 e frase 4 é  3.851623338336251


<b> Atividade </b>

7) Qual par de frase teve maior distância? E qual teve menor? Este resultado faz sentido? Explique.<br>
***R.: Considerando o método WMD O maior resultado foi entre a frase 1 e 4, e o menor representado pela frase 1 e frase 2. Considerando o resultado, a regra é quanto mais próximo do valor de zero, maior é a similaridade. O mesmo resultado entre a frase 1 e 2 já apontaram maior similaridade em outros métodos. ***

# Classificação de Documentos

A clssificação de documentos é muito útil em vários aspectos. Um dos tipos de classificação de texto é a análise de sentimentos.

A fim de ilustrar a classificação de documentos iremos criar um modelo para classificar uma frase como positiva ou negativa.

## Dataset

<b> Atividade </b>

8) Carregue o dataset com o pandas e depois dê o head no dataframe.


Link download: https://drive.google.com/open?id=15azJWdEEPGsXQGiDmEOseTBJcquWvBQc

<b> Este dataset é sobre revisões de filmes do IMDB. </b>

In [195]:
%%time
# carregar
##na variável path coloque o caminho do embedding baixado:
path = "/content/drive/MyDrive/PUC Minas/Materias/PROCESSAMENTO DE LINGUAGEM NATURAL/Aula 3/imdb-reviews-pt-br.csv"

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 28.1 µs


In [196]:
# Carregando o dataset para a variável df
# Esse dataset é do IMDB e possui várias opiniões de filmes
df = pd.read_csv(path)
df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


## Representação dos dados

O sentimento positivo e negativo iremos binarizar cada um deles. Seja 1 positivo e 0 negativo.

Iremos representar o texto de duas formas:

- Bag of Words (BOW)
- Embedding

Depois iremos comparar o resultado de cada um deles.

### Representação Target

<b> Atividade </b>

9) Faça a representação dos sentimentos. 1 positivo; 0 negativo

In [197]:
# Fiz o replace para substituir os valores 'net' e 'pos' em 0 e 1
target = df['sentiment'].replace(['neg','pos'],[0,1])

In [198]:
# Fazendo a contagem dos valores negativos e positivos do dataset
target

0        0
1        0
2        0
3        0
4        0
        ..
49454    1
49455    1
49456    1
49457    1
49458    1
Name: sentiment, Length: 49459, dtype: int64

In [253]:
# Divindindo o dataset entre treino e teste
x_train, x_teste, y_train, y_teste = train_test_split(df['text_pt'], df['sentiment'] )

In [254]:
x_train.shape

(37094,)

In [201]:
y_train.shape

(37094,)

### Bag of Words (BOW)

<b> Atividade </b>

10) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords``` no dataframe para armazenar este dado processado):

- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto sem tokenização, ou seja uma string

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [202]:
def pre_processamento_texto_return_str(corpus):
  corpus_alt = re.findall(r'\w+(?:\w+)?|[^\w\s]', corpus)
  corpus_alt = [t.lower() for t in corpus_alt]
  portugues_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  corpus_alt =[t for t in corpus_alt if t not in string.punctuation]
  corpus_alt = [unicode(t) for t in corpus_alt]
  return corpus_alt

In [271]:
df['text_pt_str'] = df['text_pt'].progress_apply(pre_processamento_texto_return_str)

  0%|          | 0/49459 [00:00<?, ?it/s]

<b> Atividade </b>

11) Aplique a representação do texto processado anteriormente com CountVectorizer.

In [272]:
x_train, x_teste, y_train, y_teste = train_test_split(df['text_pt_str'], target, random_state=123)

In [274]:
vect_bag = CountVectorizer()

In [277]:
# x_train_bow = vect_bag.fit_transform(x_train)
# 'list' object has no attribute 'lower'

A partir daqui deu o erro de 'list' object has no attribute 'lower'

### Embedding

<b> Atividade </b>

12) Aplique o pré-processamento listado abaixo na coluna ``text_pt`` (crie uma nova coluna ```text_pt_sem_stopwords_token``` no dataframe para armazenar este dado processado):

- Aplique lower
- Remova as stopwords do texto
- Remova as pontuções
- Mantenha o texto com tokenização

<b> Dica: </b> use o ```progress_apply``` para exibir a barra de progresso:

```python
from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas()
df["colunas"].progress_apply(lambda x: preprocessamento(x))
```

In [226]:
def pre_processamento_texto_return_token(corpus):
  corpus_alt = re.findall(r'\w+(?:\w+)?|[^\w\s]', corpus)
  corpus_alt = [t.lower() for t in corpus_alt]
  portugues_stops = stopwords.words('portuguese')
  corpus_alt = [t for t in corpus_alt if t not in portugues_stops]
  corpus_alt = [re.sub(r'\d','',t) for t in corpus_alt]
  corpus_alt =[t for t in corpus_alt if t not in string.punctuation]
  corpus_alt = [unicode(t) for t in corpus_alt]
  return corpus_alt

In [278]:
df['text_pt_sem_stopwords_token'] = df['text_pt'].progress_apply(lambda x: pre_processamento_texto_return_token(x))

  0%|          | 0/49459 [00:00<?, ?it/s]


<b> Atividade </b>

13) Aplique a representação do texto com Embeddings. Cada palavra tem um embedding, o embedding da frase é a média de todos embeddings.

In [279]:
x_train2, x_test2, y_train_emb, y_test_emb = train_test_split(df['text_pt_sem_stopwords_token'], target, random_state=123) 

In [280]:
df['text_pt_sem_stopwords_token']

0        [vez, sr, costner, arrumou, filme, tempo, nece...
1        [exemplo, motivo, maioria, filmes, ação, mesmo...
2        [primeiro, tudo, odeio, raps, imbecis, poderia...
3        [beatles, puderam, escrever, músicas, todos, g...
4        [filmes, fotos, latão, palavra, apropriada, ve...
                               ...                        
49454    [média, votos, baixa, fato, funcionário, locad...
49455    [enredo, algumas, reviravoltas, infelizes, ina...
49456    [espantado, forma, filme, maioria, outros, méd...
49457    [christmas, together, realmente, veio, antes, ...
49458    [drama, romântico, classe, trabalhadora, diret...
Name: text_pt_sem_stopwords_token, Length: 49459, dtype: object

In [281]:
def calcula_embedding_frase(tokens):
  return np.mean([word_vectors[t] for t in tokens if t in word_vectors.vocab], axis=0)

In [282]:
# Para cada linha percorrida será aplicado o embedding
x_train_emb = x_train2.progress_apply(calcula_embedding_frase)

  0%|          | 0/37094 [00:00<?, ?it/s]

In [283]:
# Para cada linha percorrida será aplicado o embedding
x_test_emb = x_test2.progress_apply(calcula_embedding_frase)

  0%|          | 0/12365 [00:00<?, ?it/s]

In [284]:
x_train_emb.shape

(37094,)

In [285]:
x_test_emb.shape

(12365,)

## Treinamento

### CountVectorizer


<b> Atividade </b>

14) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

```python
X_train_bow, X_test_bow, y_train_bow, y_test_bow = train_test_split(X_bag, target,random_state=123)
```

In [237]:
modelo_bow = LogisticRegression()


<b> Atividade </b>

15) Treine com uma regressão logística

In [250]:
# Fazendo o treinamento dos dados com Regressão Logística
modelo_bow = LogisticRegression()



<b> Atividade </b>

16) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

In [252]:
modelo_bow.predict(x_teste_bow)  

NameError: ignored

### Embedding


<b> Atividade </b>

17) Faça a divisão dados dados em treino e teste como no exemplo abaixo:

Verifique o shape do X treino e X teste. Caso eles estejam com apenas uma dimensão, você precisa tranformá-los para duas dimensões, caso contrário ocorrerá erro no treinamento.


<b> Atividade </b>

18) Treine com uma regressão logística


<b> Atividade </b>

19) Calcule as métricas de resultado utilizando método abaixo:

```python
print(classification_report(y_test_bow, y_pred))
```

#### Calcule as métricas


<b> Atividade </b>

20) Compare os resultados obtidos com o BagOfWords e com o Embedding. Explique os possíveis motivos desta diferença.

# Análise de sentimentos

O modelo que criamos anteriormente é para ilustrar como podemos realizar classificação de documentos.
Quando a tarefa é sobre análise de sentimentos, temos duas opções: treinar nosso próprio modelo, como feito anteriormente ou utilizar uma das inúmeras ferramentas prontas.

Vamos testar as seguintes ferramentas:

- Vader
- Textblob
- Affin

Nesta atividade iremos utilizar as duas variáveis abaixo:

In [ ]:
texto_neg = df.loc[0, "text_en"]
texto_pos = df.loc[49431, "text_en"]

## Vader

<b> Apenas Inglês </b>

O VADER (Valence Aware Dictionary e sEntiment Reasoner) é uma ferramenta de análise de sentimentos baseada em regras e léxico, especificamente identifica os sentimentos expressos nas mídias sociais.

- positive sentiment: compound score >= 0.05
- neutral sentiment: (compound score > -0.05) e (compound score < 0.05)
- negative sentiment: compound score <= -0.05

Mais informações: https://github.com/cjhutto/vaderSentiment


<b> Atividade </b>

21) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
analyzer = SentimentIntensityAnalyzer()
analyzer.polarity_scores(texto)
```

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## TextBlob

<b> Apenas inglês </b>

https://www.presentslide.in/2019/08/sentiment-analysis-textblob-library.html

<b> Atividade </b>
 
22) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
sentence=TextBlob(texto)
sentence.sentiment
```

In [ ]:
from textblob import TextBlob

## Afinn

- Valor maior que 0 indica sentimento positivo
- Valor menor que 0 indica sentimento negativo

<b> Atividade </b>

23) Aplique este método nas revisões ```texto_pos``` e ```texto_neg```.
Para aplicar:

```python
afinn = Afinn()
afinn.score(texto)
```

In [ ]:
from afinn import Afinn

<b> Atividade </b>

24) Para você, qual ferramenta teve melhor comportamento?  Justifique com exemplos.

# Dica:
## Quando for trabalhar com um dataset em inglês, a biblioteca Spacy facilita!

In [ ]:
!python -m spacy download en_core_web_md

In [ ]:
import spacy
import pandas as pd

O scpay forne um pacote que já tem série de modelos já treinados em NLP. Inclusive para os embeddings em inglês.

Para mais informações vá em:

https://spacy.io/models/en#en_core_web_md



Com o método abaixo carregamos um dos modelos do spacy:

```python
nlp = spacy.load('en_core_web_md')
```

Para aplicar o modelo, basta passar o texto para o modelo carregado anteriormente:

```python
doc = nlp("This is some text that I am processing with Spacy")
```

Carregue o modelo e imprima doc

In [ ]:
nlp = spacy.load('en_core_web_md')

In [ ]:
doc = nlp("This is some text that I am processing with Spacy")

In [ ]:
doc

Ao aplicar o modelo carregado a variável <b> doc </b> já possui os embeddings de cada uma das palavras e o embedding da frase, que é a média de todos vetores de todas palavras.

```python
#vetor da primeira palavra
doc[0].vector
#vetor agregado pela média - embedding do documento
doc.vector
```
O código abaixo mostra que a média de uma posição em específico dos embeddings de todas as palavras e a posição do embedding do documento possuem o mesmo valor.

In [ ]:
def calcula_media_posicao(x):
    soma = 0
    vector = []
    for i in range(0,len(doc)):
        vector.append(doc[i].vector)    
    
    for v in vector:
        soma += v[x]
    return soma/len(doc)

In [ ]:
round(calcula_media_posicao(10),6)

In [ ]:
round(doc.vector[10], 6)